In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['jazz']
YEARS = [2010, 2024] 

In [6]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')
print(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, LASTFM_API_KEY) 

23221855eb33466f8ddd5fc5192d4298 fea76b59ad424df587defd9ac0eb5f78 2a49373141e7a078737c0b92b60084a8


In [8]:
GENRES = ['jazz']
YEARS = [2010, 2024]

query_terms = [
    'jazz',
    'smooth jazz',
    'jazz fusion',
    'latin jazz',
    'contemporary jazz'
]

# Términos para búsqueda


# 4. Inicializa el cliente de Spotify
try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None

✅ Conexión a Spotify exitosa.


In [9]:
print(type(sp)) # Verifica el tipo del objeto sp

<class 'spotipy.client.Spotify'>


In [10]:
import time 
import pandas as pd 

def extract_spotify_data_simplificada(sp, years, query_terms): 
    """
    VERSION ESTABLE CON LÍMITES: 500 total, 250 por año. Extrae datos básicos
    y la Popularidad (sin Audio Features) para garantizar estabilidad.
    """
    
    if sp is None:
        print("Error: Cliente de Spotify no inicializado.")
        return pd.DataFrame()

    spotify_data = []
    MAX_TOTAL_SONGS = 500     # Límite global
    MAX_SONGS_PER_YEAR = 250  # Límite por año (REINTRODUCIDO)
    
    for year in years:
        
        # 1. VERIFICACIÓN DEL LÍMITE GLOBAL
        if len(spotify_data) >= MAX_TOTAL_SONGS:
            print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
            break
            
        year_range = f'{year}-{year}'
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")
        
        songs_added_in_year = 0 # Contador reiniciado para el límite anual
        
        for term in query_terms:
            
            # 2. VERIFICACIÓN DEL LÍMITE ANUAL (antes de la búsqueda)
            if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                print(f"🛑 LÍMITE ANUAL ALCANZADO ({MAX_SONGS_PER_YEAR}) para {year}. Pasando al siguiente año.")
                break # Rompe el bucle de términos, y avanza al siguiente año.
            
            query = f'{term} year:{year_range}'
            print(f"   -> Buscando con término: {term}")
            
            # --- BLOQUE DE BÚSQUEDA ---
            try:
                results = sp.search(q=query, type='track', limit=50)
            except Exception as e:
                print(f"   ❌ ERROR grave al buscar con '{term}': {e}. Esperando 5s.")
                time.sleep(5)
                continue
            
            songs_added_in_term = 0
            
            while results: 
                
                for track in results['tracks']['items']:
                    
                    track_id = track['id']
                    
                    # 3. VERIFICACIÓN DE LÍMITES Y DEDUPLICACIÓN
                    if len(spotify_data) >= MAX_TOTAL_SONGS:
                        print(f"🛑 LÍMITE GLOBAL ALCANZADO ({MAX_TOTAL_SONGS}). Terminando la extracción.")
                        return pd.DataFrame(spotify_data)
                        
                    # 4. VERIFICACIÓN DE LÍMITE ANUAL (por si se alcanza con la canción actual)
                    if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                        break # Rompe el bucle de canciones (for track...), pasará al chequeo de abajo.
                        
                    if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                        continue
                        
                    # 5. Añadir la canción
                    data_row = {
                        'Artista': track['artists'][0]['name'],
                        'Género musical': f'funk (Origen: {term})',
                        'Tipo': 'Canción',
                        'Nombre': track['name'],
                        'Año de lanzamiento': year,
                        'ID_Spotify': track_id,
                        'ID_Album': track['album']['id'],
                        'Popularidad': track.get('popularity', None) 
                    }
                    spotify_data.append(data_row)
                    songs_added_in_year += 1
                    songs_added_in_term += 1

                # Rompe el bucle de paginación si se alcanzó el límite anual en el bucle interior.
                if songs_added_in_year >= MAX_SONGS_PER_YEAR:
                    break 

                # 6. Paginación
                try:
                    if results['tracks']['next']:
                        results = sp.next(results['tracks'])
                        time.sleep(1.0) # Pausa estable entre páginas de 50
                    else:
                        results = None
                except Exception as e:
                    print(f"   ❌ ERROR al obtener la página siguiente: {e}")
                    results = None

            print(f"      -> Añadidas {songs_added_in_term} canciones ÚNICAS de '{term}'.")
            
    return pd.DataFrame(spotify_data)

In [11]:
if sp:
    # Llamamos a la versión simplificada y estable
    df_spotify_raw = extract_spotify_data_simplificada(sp, YEARS, query_terms) 
    
    print("\n--- RESULTADO FINAL DE EXTRACCIÓN ESTABLE ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}") 
    print(df_spotify_raw[['Nombre', 'Artista', 'Popularidad']].head())
else:
    print("❌ El cliente 'sp' no está inicializado. No se pudo ejecutar la extracción.")


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2010 ---
   -> Buscando con término: jazz
      -> Añadidas 49 canciones ÚNICAS de 'jazz'.
   -> Buscando con término: smooth jazz
      -> Añadidas 97 canciones ÚNICAS de 'smooth jazz'.
   -> Buscando con término: jazz fusion
      -> Añadidas 97 canciones ÚNICAS de 'jazz fusion'.
   -> Buscando con término: latin jazz
      -> Añadidas 7 canciones ÚNICAS de 'latin jazz'.
🛑 LÍMITE ANUAL ALCANZADO (250) para 2010. Pasando al siguiente año.

--- INICIANDO EXTRACCIÓN PARA EL AÑO 2024 ---
   -> Buscando con término: jazz
      -> Añadidas 43 canciones ÚNICAS de 'jazz'.
   -> Buscando con término: smooth jazz
      -> Añadidas 98 canciones ÚNICAS de 'smooth jazz'.
   -> Buscando con término: jazz fusion
      -> Añadidas 99 canciones ÚNICAS de 'jazz fusion'.
   -> Buscando con término: latin jazz
🛑 LÍMITE GLOBAL ALCANZADO (500). Terminando la extracción.

--- RESULTADO FINAL DE EXTRACCIÓN ESTABLE ---
Total de registros extraídos: 500
                No

In [12]:
# VERIFICACIÓN DE CANCIONES
print(f"Número de canciones encontradas en df_spotify_raw: {len(df_spotify_raw)}")
print(f"Primeras filas (head) de df_spotify_raw:\n{df_spotify_raw.head()}")

Número de canciones encontradas en df_spotify_raw: 500
Primeras filas (head) de df_spotify_raw:
                   Artista       Género musical     Tipo               Nombre  \
0                  Quantic  funk (Origen: jazz)  Canción  Cumbia Sobre El Mar   
1  Alessandro Alessandroni  funk (Origen: jazz)  Canción        Jazz Whistler   
2                Bop Alloy  funk (Origen: jazz)  Canción            Jazzmatic   
3             Caro Emerald  funk (Origen: jazz)  Canción    A Night Like This   
4                     Sade  funk (Origen: jazz)  Canción      Soldier of Love   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2010  61muv9lHtgN18qbm9oK9pH  0NiH3ROfMf9ZGYj07RP0B0   
1                2010  2eO48qUibU8IAM4tR51mrV  5LKhJ089woBuOf9e3eRzHC   
2                2010  1cDM4wL3FKPO4JpK3YNiWT  6X8ohJVWti6xDto1k9mFlO   
3                2010  7nuBU1HCcOwG5f1orN4ByW  7KYgUgJTrsxT4gkHRALi3R   
4                2010  0bgsj8kz5zWbQ6njk2Oqoz  3QzXO

In [13]:
# --- CELDA COMPLETA: EXTRACCIÓN Y UNIFICACIÓN DE ÁLBUMES ---

# Asegúrate de que las librerías necesarias (pandas, time) y la variable 'sp' estén disponibles
import pandas as pd
import time
# Nota: La variable 'sp' debe estar definida por la Celda 1.

# -------------------------------------------------------------------
# BLOQUE 1: DEFINICIÓN DE LA FUNCIÓN
# -------------------------------------------------------------------

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # El set ya se crea cuando se llama a la función con album_ids_list.
    unique_album_ids = album_ids
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] 
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa para no saturar la API
            
        except Exception as e:
            # Capturamos errores como álbumes que han sido eliminados de Spotify
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)

# -------------------------------------------------------------------
# BLOQUE 2: EJECUCIÓN Y FUSIÓN
# -------------------------------------------------------------------

# Verificación de que el DataFrame de canciones está disponible
if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    print("\n--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---")
    
    # 1. VERIFICACIÓN CRÍTICA: La columna 'ID_Album' debe existir.
    if 'ID_Album' not in df_spotify_raw.columns:
        print("❌ ERROR CRÍTICO: La columna 'ID_Album' NO se encontró. La Celda 3 falló al guardar el ID del álbum.")
    else:
        
        # 2. Obtener IDs únicos válidos de los álbumes
        album_ids_list = df_spotify_raw['ID_Album'].dropna().unique() 
        
        if len(album_ids_list) == 0:
            print("❌ LISTA VACÍA: Se encontraron canciones, pero 0 IDs de álbumes válidos. No se puede continuar.")
        else:
            print(f"✅ Se encontraron {len(album_ids_list)} IDs de álbumes únicos. Extrayendo ahora...")
            
            # 3. Ejecutar la función de extracción (Tardará unos minutos)
            df_album_details = extract_album_details(sp, album_ids_list)
            
            # 4. Fusionar (JOIN) los detalles del álbum con los datos de las canciones
            df_spotify_final = pd.merge(
                df_spotify_raw,
                df_album_details[['ID_Album', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']],
                on='ID_Album',
                how='left' 
            )
            
            print(f"\n--- PROCESO DE ÁLBUMES COMPLETADO ---")
            print(f"DataFrame intermedio 'df_spotify_final' creado con {len(df_spotify_final)} filas.")
            print(df_spotify_final.head())
            
else:
    print("❌ ERROR: El DataFrame de canciones (df_spotify_raw) no está disponible. Vuelve a ejecutar la Celda 3.")


--- INICIANDO PROCESO DE UNIFICACIÓN DE ÁLBUMES ---
✅ Se encontraron 298 IDs de álbumes únicos. Extrayendo ahora...

--- INICIANDO EXTRACCIÓN DE METADATOS DE 298 ÁLBUMES ---
✅ Extracción de 298 álbumes completada.

--- PROCESO DE ÁLBUMES COMPLETADO ---
DataFrame intermedio 'df_spotify_final' creado con 500 filas.
                   Artista       Género musical     Tipo               Nombre  \
0                  Quantic  funk (Origen: jazz)  Canción  Cumbia Sobre El Mar   
1  Alessandro Alessandroni  funk (Origen: jazz)  Canción        Jazz Whistler   
2                Bop Alloy  funk (Origen: jazz)  Canción            Jazzmatic   
3             Caro Emerald  funk (Origen: jazz)  Canción    A Night Like This   
4                     Sade  funk (Origen: jazz)  Canción      Soldier of Love   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2010  61muv9lHtgN18qbm9oK9pH  0NiH3ROfMf9ZGYj07RP0B0   
1                2010  2eO48qUibU8IAM4tR51mrV  5LKh

In [14]:
def extract_lastfm_artist_details(artist_name):
    """Extrae biografía y estadísticas de popularidad de un artista."""
    
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if 'artist' in data:
            artist_info = data['artist']
            
            # Extraer los datos relevantes (Biografía, Popularidad/Plays)
            biography = artist_info.get('bio', {}).get('summary', 'N/A')
            playcount = artist_info.get('stats', {}).get('playcount', '0')
            listeners = artist_info.get('stats', {}).get('listeners', '0')
            
            return {
                'Artista': artist_name,
                'Biografia_Resumen': biography,
                'Playcount_LastFM': playcount,
                'Listeners_LastFM': listeners,
            }
    except Exception:
        # En caso de error de conexión o artista no encontrado, devuelve None
        pass 
        
    return None

# --- Ejecución para Artistas Únicos ---

if 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    
    # Obtener la lista única de artistas para evitar búsquedas repetidas
    unique_artists = df_spotify_raw['Artista'].unique()
    lastfm_results = []
    
    print("\n--- EXTRACCIÓN DETALLES LAST.FM ---")
    
    for artist in unique_artists: 
        details = extract_lastfm_artist_details(artist)
        if details:
            lastfm_results.append(details)
        time.sleep(0.5) # Pausa crucial para no sobrecargar la API

    df_lastfm = pd.DataFrame(lastfm_results)
    
    # 5. UNIFICACIÓN DE DATOS
    # Fusionar los datos de Spotify con los detalles de Last.fm
    df_final_jazz    = pd.merge(
        df_spotify_raw, 
        df_lastfm[['Artista', 'Playcount_LastFM', 'Listeners_LastFM']],
        on='Artista', 
        how='left' # Usamos left join para mantener todas las canciones de Spotify
    )

    print("\n--- DATAFRAME FINAL DE JAZZ LISTO PARA ALMACENAMIENTO ---")
    print(df_final_jazz.head())
    print(f"Total de registros finales para jazz: {len(df_final_jazz)}")


--- EXTRACCIÓN DETALLES LAST.FM ---

--- DATAFRAME FINAL DE JAZZ LISTO PARA ALMACENAMIENTO ---
                   Artista       Género musical     Tipo               Nombre  \
0                  Quantic  funk (Origen: jazz)  Canción  Cumbia Sobre El Mar   
1  Alessandro Alessandroni  funk (Origen: jazz)  Canción        Jazz Whistler   
2                Bop Alloy  funk (Origen: jazz)  Canción            Jazzmatic   
3             Caro Emerald  funk (Origen: jazz)  Canción    A Night Like This   
4                     Sade  funk (Origen: jazz)  Canción      Soldier of Love   

   Año de lanzamiento              ID_Spotify                ID_Album  \
0                2010  61muv9lHtgN18qbm9oK9pH  0NiH3ROfMf9ZGYj07RP0B0   
1                2010  2eO48qUibU8IAM4tR51mrV  5LKhJ089woBuOf9e3eRzHC   
2                2010  1cDM4wL3FKPO4JpK3YNiWT  6X8ohJVWti6xDto1k9mFlO   
3                2010  7nuBU1HCcOwG5f1orN4ByW  7KYgUgJTrsxT4gkHRALi3R   
4                2010  0bgsj8kz5zWbQ6njk2Oqoz  3QzXO

In [16]:
# --- FUSIÓN FINAL SPOTIFY (CON ÁLBUMES) + LAST.FM Y GUARDADO CSV ---

import pandas as pd

# Usaremos df_spotify_final si ya lo tienes, porque incluye álbumes + Popularidad
base_df = None
if 'df_spotify_final' in locals() and not df_spotify_final.empty:
    base_df = df_spotify_final
elif 'df_spotify_raw' in locals() and not df_spotify_raw.empty:
    base_df = df_spotify_raw

if base_df is not None and 'df_lastfm' in locals():
    print("✅ DataFrames disponibles para fusión final.")
    print("Columnas base (Spotify):", base_df.columns.tolist())
    print("Columnas Last.fm:", df_lastfm.columns.tolist())

    df_proyecto_final = pd.merge(
        base_df,
        df_lastfm,
        on='Artista',
        how='left'
    )

    print("Columnas después de la fusión final:", df_proyecto_final.columns.tolist())

    # Limpieza de Last.fm
    if 'Biografia_Resumen' in df_proyecto_final.columns:
        df_proyecto_final['Biografia_Resumen'] = df_proyecto_final['Biografia_Resumen'].fillna('N/A')

    for col in ['Playcount_LastFM', 'Listeners_LastFM']:
        if col in df_proyecto_final.columns:
            df_proyecto_final[col] = df_proyecto_final[col].fillna(0).astype(int)

    # Guardar CSV definitivo
    output_filename = "data_jazz_final2.csv"
    df_proyecto_final.to_csv(output_filename, index=False, encoding="utf-8")

    print(f"\n🎉 CSV FINAL GENERADO CON ÉXITO 🎉")
    print(f"Archivo guardado como: {output_filename}")
    print(f"Filas totales: {len(df_proyecto_final)}")

    columnas_demo = [c for c in ['Nombre','Artista','Popularidad','Listeners_LastFM','Biografia_Resumen'] if c in df_proyecto_final.columns]
    if columnas_demo:
        print("\n--- Primeras 5 filas ---")
        print(df_proyecto_final[columnas_demo].head())
    else:
        print("⚠️ No se encontraron las columnas esperadas para el ejemplo de head().")
else:
    print("❌ Falta base_df (Spotify) o df_lastfm; ejecuta antes las celdas de extracción.")

✅ DataFrames disponibles para fusión final.
Columnas base (Spotify): ['Artista', 'Género musical', 'Tipo', 'Nombre', 'Año de lanzamiento', 'ID_Spotify', 'ID_Album', 'Popularidad', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album']
Columnas Last.fm: ['Artista', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']
Columnas después de la fusión final: ['Artista', 'Género musical', 'Tipo', 'Nombre', 'Año de lanzamiento', 'ID_Spotify', 'ID_Album', 'Popularidad', 'Nombre_Album', 'Año_Lanzamiento_Album', 'Artista_Principal_Album', 'Biografia_Resumen', 'Playcount_LastFM', 'Listeners_LastFM']

🎉 CSV FINAL GENERADO CON ÉXITO 🎉
Archivo guardado como: data_jazz_final2.csv
Filas totales: 500

--- Primeras 5 filas ---
                Nombre                  Artista  Popularidad  \
0  Cumbia Sobre El Mar                  Quantic           60   
1        Jazz Whistler  Alessandro Alessandroni           32   
2            Jazzmatic                Bop Alloy           38   
3    